# Project Phase 2: Video Dialog 

## Imports

In [ ]:
import json
from pprint import pprint

#Open Search
from opensearchpy import OpenSearch

#Embeddings neighborhood
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pickle
import spacy

#Contextual embeddings and self-attention
import numpy as np
from sklearn.decomposition import PCA
from transformers import AutoConfig, AutoModelForSequenceClassification
from bertviz import model_view, head_view

# Get the interactive Tools for Matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

## 2.2 Text-based Search

### Load the video captions

In [ ]:
def load_captions_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    processed = {}
    for video_id, captions in data.items():
        processed[video_id] = {
            "segments": captions['segments'] if 'segments' in captions else captions,
        }
    return processed

# Load the data
val_data1 = load_captions_data('captions/val_1.json')
val_data2 = load_captions_data('captions/val_2.json')

# Combine dictionaries (preserving video_id as keys)
all_captions_data = {**val_data1, **val_data2}

pprint(f"Number of captions: {len(all_captions_data)}")
pprint(f"Example Captions: {all_captions_data}")

### Load the videos

In [ ]:
with open('activity_net.v1-3.min.json', 'r') as json_data:
    data = json.load(json_data)

database = {}

for video_id in data['database']:
    database["v_" + video_id] = data['database'][video_id]

# Create the list with all data, sorted by the number of annotations
sorted_database = sorted(
    database.items(),
    key=lambda x: len(x[1]['annotations']),
    reverse=True
)

# Top 10 videos in number of annotations
top_videos = dict(sorted_database[:27])

pprint(top_videos)

In [ ]:
matching_ids = set(database.keys()) & set(all_captions_data.keys())
print(f"Número de IDs correspondentes: {len(matching_ids)}")
print(f"IDs no top_videos: {list(top_videos.keys())[:5]}...")
print(f"IDs em all_captions_data: {list(all_captions_data.keys())[:5]}...")

### Compute the final captions dataset

In [ ]:
final_dataset_captions = {}


# Check and store the captions' of the top 10 videos
for video_id in top_videos:
    try:
        if (all_captions_data[video_id] != None):
            final_dataset_captions[video_id] = all_captions_data[video_id]
    except Exception as e:
        None

final_dataset_captions.pop("v_PJ72Yl0B1rY", None) # This video has no available URL

pprint(final_dataset_captions)
pprint(len(final_dataset_captions))

### Keyframe extraction

In [ ]:
import av
import av.datasets

content = av.datasets.curated("pexels/time-lapse-video-of-night-sky-857195.mp4")
with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    for i, frame in enumerate(container.decode(stream)):
        print(frame)
        frame.to_image().save(f"night-sky.{i:04d}.jpg", quality=80)

### OpenSearch connection settings

In [ ]:
#Connections to the Open Search Server
host = 'api.novasearch.org'
port = 443

user = ''
password = ''
index_name = user

Test if OpenSearch is up and running

In [ ]:
# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    use_ssl = True,
    url_prefix = 'opensearch_v2',
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")

In [ ]:
client.indices.delete(index=index_name, ignore=[400, 404])

### Create the index mappings for video captions

In [ ]:
index_body = {
    "settings": {
        "index": {
            "number_of_replicas": 0,
            "number_of_shards": 4,
            "refresh_interval": "-1",
            "knn": "true"
        },
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            #video_id
            "title": {
                "type": "keyword"
            },
            #sentences
            "description": {
                "type": "text"
            },
        }
    }
}

if client.indices.exists(index=index_name):
    print("Index already exists.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index...')
    print(response)

Index the dataset

In [ ]:
for video_id, data in final_dataset_captions.items():
    print(f"Title: {video_id}")
    print(f"Description: {data['segments']['sentences']}")

for video_id, data in final_dataset_captions.items():
    filtered_caption = {
        "title": video_id,
        "description": data['segments']['sentences']
    }
    
    resp = client.index(index=index_name, id=video_id, body=filtered_caption)
    print(resp['result'])

### Search Functionality

Text-based Search 

In [ ]:
client.indices.refresh(index=index_name)

In [ ]:
client.count(index = index_name)

In [ ]:
# Search for a specific caption

qtxt = "finally gets a spare."

text_query = {
  "size": 5,
  "_source": ['title', 'description'],
  "query": {
    "multi_match": {
      "query": qtxt,
      "fields": ['description'],
    }
  }
}

response = client.search(
    body=text_query,
    index=index_name
)

print("\nSearch results:")
pprint(response)

Term-level Search

In [ ]:
# Search for a specific video ID

term_query = {
    "size": 5,
    "_source": ["title", "description"],
    "query": {
        "term": {
            "title": "v_od9EdcDcByA"
        }
    }
}

response = client.search(
    body=term_query,
    index=index_name
)

print("\nSearch results:")
pprint(response)

Boolean Search

In [ ]:
# Search for a video containing the words "skate" and "tricks"

bool_query = {
    "size": 5,
    "_source": ["title", "description"],
    "query": {
        "bool": {
            "must": [
                {"match": {"description": "skate"}},  # Must contain "skate"
                {"match": {"description": "tricks"}}  # Must contain "tricks"
            ],
        }
    }
}

response = client.search(
    body=bool_query,
    index=index_name
)

print("\nSearch results:")
pprint(response)

## 2.3 Embeddings Neighborhood

In [ ]:
client.indices.delete(index=index_name, ignore=[400, 404])

 ### Index mappings to support k-nn vectors

In [ ]:
index_body = {
    "settings": {
        "index": {
            "number_of_replicas": 0,
            "number_of_shards": 4,
            "refresh_interval": "-1",
            "knn": True
        }
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            "title": { "type": "keyword" },
            "description": { "type": "text" },
            "sentence_embedding": {
                "type": "knn_vector",
                "dimension": 768,
                "method": {
                    "name": "hnsw",
                    "space_type": "innerproduct",
                    "engine": "faiss",
                    "parameters": {
                        "ef_construction": 256,
                        "m": 48
                    }
                }
            }
        }
    }
}


if client.indices.exists(index=index_name):
    print("Index already existed. You may force the new mappings.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)

In [ ]:
client.indices.refresh(index=index_name)

In [ ]:
client.count(index = index_name)

### Dual-Encoders

In [ ]:
# Model and Tokenizer utilized for the enconding and creation of the embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

Compute the video text embeddings

In [ ]:
all_embeddings = {}

for video_id, data in final_dataset_captions.items():
    # Join all sentences to one paragraph-like string
    full_description = " ".join(data['segments']['sentences'])
    
    embedding = encode(full_description)
    
    all_embeddings[video_id] = {
        "title": video_id,
        "description": data['segments']['sentences'],
        "sentence_embedding": embedding[0].numpy()
    }
        
    resp = client.index(index=index_name, id=video_id, body=all_embeddings[video_id])
    print(resp['result'])

    stored = client.get(index=index_name, id=video_id)
    print("\nIndexed Document:")
    pprint(stored["_source"])
    print("-" * 50)

# Save the data to the pickle file
with open('all_embeddings.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)

### Load the data from the pickle file

In [ ]:
with open('all_embeddings.pkl', 'rb') as f:
    all_embeddings = pickle.load(f)
print(all_embeddings)


### Search query for embedding index

In [ ]:
client.indices.refresh(index=index_name)

In [ ]:
# Search for a specific caption using KNN
query = "finally gets a spare."
query_emb = encode(query)

query_denc = {
  'size': 5,
  '_source': ['title', 'description'],
   "query": {
        "knn": {
          "sentence_embedding": {
            "vector": query_emb[0].numpy(),
            "k": 2
          }
        }
      }
}

response = client.search(
    body = query_denc,
    index = index_name
)

print('\nSearch results:')
pprint(response)

## 2.4 Constrained Embedding Searches

In [ ]:
# Embedding of a query and search in the index using that encoded string 
nlp = spacy.load("en_core_web_sm")
query = "show me moments of a man surfing."
query_embedding = encode(query)[0].numpy().tolist()

def get_nouns_verbs(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.pos_ in {"NOUN", "VERB"}])

filter_terms = get_nouns_verbs(query)
print(f"Filter terms: {filter_terms}")

search_body = {
    'size': 100,
    "query": {
        "bool": {
            "must": [{
                "knn": {
                    "sentence_embedding": {
                        "vector": query_embedding,
                        "k": 1
                    }
                }
            }],
            "filter": [{
                "match": {
                    "description": {
                        "query": filter_terms,
                        "operator": "or"
                    }
                }
            }]
        }
    },
    "_source": ["title", "description"]
}

response = client.search(index=index_name, body=search_body)
print(f"Total documents found: {response['hits']['total']['value']}")

for hit in response["hits"]["hits"]:
    print(f"Title: {hit['_source']['title']}")
    print("Relevant sentences:")
    for sentence in hit['_source']['description']:
        if "surf" in sentence.lower():
            print(f"  → {sentence}")
        else:
            print(f"  - {sentence}")


In [ ]:
query = "bokling"  # Intentional typo for "bowling"

search_body = {
    "query": {
        "match": {
            "description": {
                "query": query,
                "fuzziness": "AUTO",  # Automatically corrects "bokling" to "bowling"
                "operator": "or"
            }
        }
    }
}

response = client.search(index=index_name, body=search_body)
print(f"Total documents found: {response['hits']['total']['value']}")

for hit in response["hits"]["hits"]:
    print(f"Title: {hit['_source']['title']}")
    print("Full description:")
    for sentence in hit['_source']['description']:
        print(f"  - {sentence}")


## 2.5 Contextual embeddings and Self-attention

### Contextual embeddings

In [ ]:
sentence = "He raises his hands feeling victorious."

In [ ]:
# Load a pre-trained BERT model fine-tuned on MS MARCO for relevance ranking
model_path = 'nboost/pt-bert-base-uncased-msmarco'

# Initialize tokenizer and model with specific configurations
tokenizer = AutoTokenizer.from_pretrained(model_path)  # Converts text to token IDs
config = AutoConfig.from_pretrained(
    model_path,
    output_hidden_states=True,  # Returns all hidden states for analysis
    output_attentions=True      # Returns attention weights for interpretability
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    config=config  # Load model with the specified configuration
)

# Tokenize input sentence with proper formatting for BERT
inputs = tokenizer(
    sentence,
    return_tensors='pt',         # Returns PyTorch tensors
    add_special_tokens=True,     # Adds [CLS] and [SEP] tokens
    max_length=512,              # Truncates to BERT's max length
    padding=True,                # Pads sequences to max length
    truncation=True              # Truncates longer sequences
)

# Convert token IDs back to tokens for interpretability
input_ids = inputs["input_ids"][0]
input_tokens_list = tokenizer.convert_ids_to_tokens(input_ids)

# Run model inference without gradient calculation (faster, less memory)
with torch.no_grad():
    outputs = model(**inputs)  # Returns logits, hidden_states, attentions

Hidden layer embedding

In [ ]:
# The format is as follow:
# outputs['hidden_states'][layer_m][0][token_n]
layer_m = 12
token_n = 1

# Get all the embeddings of one layer:
output_embeddings = outputs['hidden_states'][layer_m][0]
output_embeddings.shape

hidden_states = outputs.hidden_states

In [ ]:
def display_scatterplot(data, words):

    if data.shape[1] == 2:
        twodim = data
    else:
        pca = PCA()
        pca.fit(output_embeddings.detach().numpy())
        twodim = pca.transform(data)[:,:2]
    
    plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

    return

display_scatterplot(output_embeddings.detach().numpy(), input_tokens_list)

In [ ]:
rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*4)
fig.set_figwidth(cols*4+3)
plt.rcParams.update({'font.size': 6})


layer = 0
for r in range(rows):
    for c in range(cols):
       
        ax = ax_full[r,c]
        
        plt.rcParams.update({'font.size': 10})
        current_hidden_state = hidden_states[layer][0].detach().numpy()
        
        if current_hidden_state.shape[1] == 2:
            twodim = current_hidden_state
        else:
            twodim = PCA().fit_transform(current_hidden_state)[:,:2]

        plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
        im = ax.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
        for word, (x,y) in zip(input_tokens_list, twodim):
            ax.text(x+0.05, y+0.05, word[1:])
        
        # Show all ticks and label them with the respective list entries
        ax.set_title("Layer " + str(layer))
            
        # Loop over data dimensions and create text annotations.
        layer = layer + 1

fig.suptitle("Visualization of all output embeddings from all layers")
plt.show()

### Positional embeddings

In [ ]:
sentence1 = "hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello"
sentence2 = "bye bye bye bye bye bye bye bye bye bye bye bye bye bye bye bye bye bye bye bye"

In [ ]:
model_path = 'nboost/pt-bert-base-uncased-msmarco'

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

inputs = tokenizer(sentence1, sentence2, return_tensors='pt', add_special_tokens=True, max_length=512, padding=True, truncation=True)
input_ids = inputs["input_ids"][0]
input_tokens_list = tokenizer.convert_ids_to_tokens(input_ids)
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# The format is as follow:
# outputs['hidden_states'][layer_m][0][token_n]
layer_m = 12
token_n = 1
# Get all the embeddings of one layer:
output_embeddings = outputs['hidden_states'][layer_m][0]
output_embeddings.shape

hidden_states = outputs.hidden_states

In [ ]:
def display_scatterplot(data, words):

    if data.shape[1] == 2:
        twodim = data
    else:
        pca = PCA()
        pca.fit(output_embeddings.detach().numpy())
        twodim = pca.transform(data)[:,:2]
    
    plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

    return

display_scatterplot(output_embeddings.detach().numpy(), input_tokens_list)

In [ ]:
layer = 1

rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*4)
fig.set_figwidth(cols*4+3)
plt.rcParams.update({'font.size': 6})

# Define your axis scale ranges
x_range = (-12, 12)
y_range = (-6, 6)

layer = 0
for r in range(rows):
    for c in range(cols):
       
        ax = ax_full[r, c]
        plt.rcParams.update({'font.size': 10})

        current_hidden_state = hidden_states[layer][0].detach().numpy()

        # Filter out [CLS] and [SEP]
        tokens_filtered = []
        vectors_filtered = []
        for i, token in enumerate(input_tokens_list):
            if token not in ['[CLS]', '[SEP]']:
                tokens_filtered.append(token)
                vectors_filtered.append(current_hidden_state[i])

        vectors_filtered = np.array(vectors_filtered)

        # Apply PCA if necessary
        if vectors_filtered.shape[1] == 2:
            twodim = vectors_filtered
        else:
            twodim = PCA().fit_transform(vectors_filtered)[:, :2]

        plt.style.use('default')
        ax.scatter(twodim[:, 0], twodim[:, 1], edgecolors='k', c='r')

        for word, (x, y) in zip(tokens_filtered, twodim):
            ax.text(x + 0.05, y + 0.05, word)

        # Set title and fixed axes
        ax.set_title("Layer " + str(layer))
        ax.set_xlim(x_range)
        ax.set_ylim(y_range)

        layer += 1

fig.suptitle("Visualization of all output embeddings from all layers (uniform axis scale)")
plt.tight_layout()
plt.show()




### Self-attention

Cross-encoder

In [ ]:
sentence = "He raises his hands feeling victorious."
question = "What did he raise?"

In [ ]:
model_path = 'nboost/pt-bert-base-uncased-msmarco'

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModel.from_pretrained(model_path, config=config)

inputs = tokenizer(sentence, question, return_tensors='pt', add_special_tokens=True, max_length=512, padding=True, truncation=True)
input_ids = inputs["input_ids"][0]
tokens = tokenizer.convert_ids_to_tokens(input_ids)

with torch.no_grad():
    outputs = model(**inputs)

attentions = outputs.attentions

Visualization of all the attention heads in one layer

In [ ]:
sep_indices = [i for i, token in enumerate(tokens) if token == '[SEP]']

layer = 11

rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*6)
fig.set_figwidth(cols*6+4)
plt.rcParams.update({'font.size': 10})

j = 0
for r in range(rows):
    for c in range(cols):
        ax = ax_full[r, c]

        sattention = attentions[layer][0][j].numpy()
        sattention = np.flip(sattention, 0)  # flip vertically to match y-axis

        im = ax.pcolormesh(sattention, cmap='gnuplot')

        ax.set_title(f"Head {j}")
        ax.set_yticks(np.arange(len(tokens)))
        if c == 0:
            ax.set_yticklabels(reversed(tokens))
            ax.set_ylabel("Queries")
        else:
            ax.set_yticks([])

        ax.set_xticks(np.arange(len(tokens)))
        if r == rows - 1:
            ax.set_xticklabels(tokens)
            ax.set_xlabel("Keys")
            plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        else:
            ax.set_xticks([])

        # Add vertical and horizontal lines at [SEP] positions
        for sep in sep_indices:
            # Since we flipped the matrix vertically, y index needs to be (len - sep - 1)
            ax.axvline(sep, color='white', linestyle='--', linewidth=1)
            ax.axhline(len(tokens) - sep - 1, color='white', linestyle='--', linewidth=1)

        j += 1

fig.suptitle("Layer " + str(layer) + " Multi-head Self-attentions")
cbar = fig.colorbar(im, ax=ax_full, location='right', shrink=0.5)
cbar.ax.set_ylabel("Selt-attention", rotation=-90, va="bottom")
plt.show()

#### Dual-encoder

In [ ]:
# We use Cross-Encoder two times, one for each sentence, and then we compare the similarities between the words
inputs_a = tokenizer(question, return_tensors="pt")
inputs_b = tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    output_a = model(**inputs_a).last_hidden_state.squeeze(0)
    output_b = model(**inputs_b).last_hidden_state.squeeze(0) 

output_a_norm = torch.nn.functional.normalize(output_a, p=2, dim=1)
output_b_norm = torch.nn.functional.normalize(output_b, p=2, dim=1)

similarity_matrix = torch.matmul(output_a_norm, output_b_norm.T)  # (seq_len_a, seq_len_b)

tokens_a = tokenizer.convert_ids_to_tokens(inputs_a["input_ids"][0])
tokens_b = tokenizer.convert_ids_to_tokens(inputs_b["input_ids"][0])

plt.figure(figsize=(10, 6))
sns.heatmap(similarity_matrix.numpy(), xticklabels=tokens_b, yticklabels=tokens_a, cmap="viridis", annot=True, fmt=".2f")
plt.xlabel("Sentence Tokens")
plt.ylabel("Question Tokens")
plt.title("Similarity between Question and Sentence Tokens")
plt.show()

### Interpretability

In [ ]:
layer = 11  # analyze the final layer
attentions = outputs.attentions  # shape: (num_layers, batch, num_heads, seq_len, seq_len)

# Get shape info
num_layers = len(attentions)
seq_len = attentions[0].shape[-1]

# Store total attention received for each token across all heads, per layer
attention_received = torch.zeros((num_layers, seq_len))

# Loop through each layer
for l in range(num_layers):
    attn = attentions[l][0]  # shape: (num_heads, seq_len, seq_len)
    attn_sum = attn.sum(dim=0)  # sum over query dimension → shape: (seq_len, seq_len)
    attention_received[l] = attn_sum.sum(dim=0)  # sum over key dimension


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

plt.figure(figsize=(10, 6))
for token_idx in range(seq_len):
    plt.plot(range(num_layers), attention_received[:, token_idx], label=tokens[token_idx])

plt.title("Total Attention Received by Each Token Across Layers")
plt.xlabel("Layer")
plt.ylabel("Total Attention Received")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()